In [1]:
# imports
import pandas as pd
import numpy
import numpy as np
import csv
import Bio
from Bio.Data import CodonTable
from Bio.Seq import Seq
import time
import numpy as np
from Bio.Data.CodonTable import unambiguous_dna_by_id
from itertools import chain
from Bio.SeqUtils.CodonUsage import CodonsDict
from Bio.Data import CodonTable
from collections import Counter
import re
import gzip
from Bio import SeqIO
import pandas as pd
from os import chdir
from statistics import mean, pstdev
from scipy.stats.mstats import gmean
from scipy.spatial import distance
from scipy.stats.stats import pearsonr 
from scipy import stats
from scipy.stats import ks_2samp
import math

In [2]:
df = pd.read_csv("seq.csv")
gene_sequence = df['seq']
df2 = pd.read_csv("max_5_ind.csv")
GFP_seq = 'atgtccaagggtgaagagctatttactggggttgtacccattttggtagaactggacggagatgtaaacggacataaattctctgttagaggtgagggcgaaggcgatgccaccaatggtaaattgactctgaagtttatatgcactacgggtaaattacctgttccttggccaaccctagtaacaactttgacatatggtgttcaatgtttctcaagatacccagaccatatgaaaaggcatgatttctttaaaagtgctatgccagaaggctacgtgcaagagagaactatctcctttaaggatgacggtacgtataaaacacgagcagaagtgaaattcgaaggggatacactagttaatcgcatcgaattaaagggtatagactttaaggaagatggtaatattctcggccataaacttgagtataatttcaactcgcataatgtgtacattacagctgacaaacaaaagaacggaattaaagcgaattttaaaatcaggcacaacgtcgaagatgggtctgttcaacttgccgatcattatcagcaaaacacccctattggtgatggtccagtcttgttacccgataatcactacttaagcacacagtctagattgtcaaaagatccgaatgaaaagcgtgatcacatggttttattggaatttgtcaccgctgcaggaataactcacggaatggacgagctttataagggatcc'
mCherry_seq = 'atggtgagcaagggcgaggaggataacatggccatcatcaaggagttcatgcgcttcaaggtgcacatggagggctccgtgaacggccacgagttcgagatcgagggcgagggcgagggccgcccctacgagggcacccagaccgccaagctgaaggtgaccaagggtggccccctgcccttcgcctgggacatcctgtcccctcagttcatgtacggctccaaggcctacgtgaagcaccccgccgacatccccgactacttgaagctgtccttccccgagggcttcaagtgggagcgcgtgatgaacttcgaggacggcggcgtggtgaccgtgacccaggactcctccctgcaggacggcgagttcatctacaaggtgaagctgcgcggcaccaacttcccctccgacggccccgtaatgcagaagaagaccatgggctgggaggcctcctccgagcggatgtaccccgaggacggcgccctgaagggcgagatcaagcagaggctgaagctgaaggacggcggccactacgacgctgaggtcaagaccacctacaaggccaagaagcccgtgcagctgcccggcgcctacaacgtcaacatcaagttggacatcacctcccacaacgaggactacaccatcgtggaacagtacgaacgcgccgagggccgccactccaccggcggcatggacgagctgtacaag'

In [3]:
def gene_seq_2_aa(gene_seq):
    gene_seq_aa = Seq(gene_seq)
    gene_seq_aa = gene_seq_aa.translate()
    gene_seq_aa = str (gene_seq_aa)
    return gene_seq_aa

def genes_seq_2_aa(genes_seq):
    genes_seq_aa = []
    for i in range(0,len(genes_seq)):
        genes_seq_aa.append(gene_seq_2_aa(genes_seq[i]))
    return genes_seq_aa

In [4]:
#finds the longest common string of S and T 
def lcs(S,T):
    m = len(S)
    n = len(T)
    counter = [[0]*(n+1) for x in range(m+1)]
    longest = 0
    lcs_set = set()
    for i in range(m):
        for j in range(n):
            if S[i] == T[j]:
                c = counter[i][j] + 1
                counter[i+1][j+1] = c
                if c > longest:
                    lcs_set = set()
                    longest = c
                    lcs_set.add(S[i-c+1:i+1])
                elif c == longest:
                    lcs_set.add(S[i-c+1:i+1])
    x = list(lcs_set)
    if len(x) == 0:
        return (0,'')
    else:
        return (len(x[0]))

In [5]:
# returns the length of the longets coomon string of the target gene and an essential gene
# call this function with: LCW(target_gene,gene_sequence[i])
def LCW(target_gene,gene_seq): 
    return lcs(gene_seq_2_aa(target_gene),gene_seq_2_aa(gene_seq))

In [6]:
# given a sequence the function returns its GC content
def gc_content(gene_seq):
    x = gene_seq.upper()
    c = x.count('C')
    g = x.count('G')
    gc_total = g+c
    gc_content = (gc_total/len(x))
    return gc_content

In [7]:
# the function calculates the gc_content profile in a window of 30 nucletides each time (300 nt window in tot).
# if the sequence length is less than 300 the rest of the profile will be filll with the average of the gc content of existing windows 
# ind -1 for 300 nt from the beginning of the sequenece (for essential genes) or 1 for 300 until the end of the sequenece (for target gene)
def gc_content_profile(gene_seq,ind):
    gc_content_profile = []
    i = 0
    if len(gene_seq)>299:
        if ind == -1:
            wind = gene_seq[0:300]
            while i < len(wind)-1:
                gc_content_profile.append(gc_content(wind[i:i+30]))
                i = i + 30
        if ind == 1:
            wind = gene_seq[len(gene_seq)-300:]
            while i < len(wind)-1:
                gc_content_profile.append(gc_content(wind[i:i+30]))
                i = i + 30
    else:
        if ind == -1:
            x = math.floor(len(gene_seq)/30)
            j = 0
            for i in range(0,x):
                gc_content_profile.append(gc_content(gene_seq[j:j+30]))
                j = j + 30
            m = mean(gc_content_profile)
            for i in range(x,10):
                gc_content_profile.append(m)
        if ind == 1:
            x = math.floor(len(gene_seq)/30)
            j = 0
            d = len(gene_seq) - (x*30)
            wind = gene_seq[d:]
            for i in range(0,x):
                gc_content_profile.append(gc_content(wind[j:j+30]))
                j = j + 30
            m = mean(gc_content_profile)
            for i in range(x,10):
                gc_content_profile.append(m)        
    return gc_content_profile

In [8]:
# calculates ditances of gc content profiles from target gene to essential gene, and to its closest gene
# returns 6 distances from essential gene and 6 from its closet genes (from the avergae on their profiles)
# paramters required: essential gene sequence[i], target gene sequence, df2.iloc[i] (the coresponding row in df2 of the chosen essential gene),all yeast genes sequence
# _essen mean a distance from eseential gene
# _close mean a distance from eseential gene closest genes

def calc_dist_gc(gene_sequence,target_gene,df2,genes_sequence):
    
    gc_target_gene = np.asarray(gc_content_profile(target_gene,1))
    gc_gene = np.asarray(gc_content_profile(gene_sequence,-1))
    
    euc_dist_gc_essen = distance.euclidean(gc_target_gene,gc_gene)
    cart_d_gc_essen = sum(gc_target_gene*gc_gene)
    corr_p_gc_essen = pearsonr(gc_target_gene,gc_gene)[0]
    corr_s_gc_essen = stats.spearmanr(gc_target_gene,gc_gene)[0]
    L1_dist_gc_essen = sum(np.abs(gc_target_gene-gc_gene))
    ks_dist_gc_essen = ks_2samp(gc_target_gene,gc_gene)[0]
    
    max_5_ind_1 = df2['max_5_ind_1']
    max_5_ind_2 = df2['max_5_ind_2']
    max_5_ind_3 = df2['max_5_ind_3']
    max_5_ind_4 = df2['max_5_ind_4']
    max_5_ind_5 = df2['max_5_ind_5']
    
    if max_5_ind_1 == -2:
        euc_dist_gc_close = []
        corr_p_gc_close = []
        corr_s_gc_close = []
        ks_dist_gc_close = []
        cart_d_gc_close = []
        L1_dist_gc_close = []
    else:
        ind = [max_5_ind_1,max_5_ind_2,max_5_ind_3,max_5_ind_4,max_5_ind_5]
        for index in ind:
            if index==-2:
                ind.remove(index)
        close_genes_gc = []
        for j in range(0,len(ind)):
            index = ind[j]
            close_genes_gc.append(gc_content_profile(genes_sequence[index],-1))
        m_close_genes_gc = np.mean(close_genes_gc,axis=0)
    
        euc_dist_gc_close = distance.euclidean(gc_target_gene,m_close_genes_gc)
        cart_d_gc_close = sum(gc_target_gene*m_close_genes_gc)
        corr_p_gc_close = pearsonr(gc_target_gene,m_close_genes_gc)[0]
        corr_s_gc_close = stats.spearmanr(gc_target_gene,m_close_genes_gc)[0]
        L1_dist_gc_close = sum(np.abs(gc_target_gene-m_close_genes_gc))
        ks_dist_gc_close = ks_2samp(gc_target_gene,m_close_genes_gc)[0]
         
    return(euc_dist_gc_essen,corr_p_gc_essen,corr_s_gc_essen,ks_dist_gc_essen,cart_d_gc_essen,L1_dist_gc_essen,euc_dist_gc_close,corr_p_gc_close,corr_s_gc_close,ks_dist_gc_close,cart_d_gc_close,L1_dist_gc_close)  

In [9]:
def altcodons(codon, table):
    """List codons that code for the same aminonacid / are also stop. 
    Given a codon, the output is a list of the alternative (synonymous) codons.

    @param codon
    @table code table id (1 is standard)
    @return list of codons

    """
    tab = unambiguous_dna_by_id[table]

    if codon in tab.stop_codons:
        return tab.stop_codons

    try:
        aa = tab.forward_table[codon]
    except:
        return []

    return [k for (k, v) in tab.forward_table.items()
            if v == aa]


In [10]:
def degeneration(codon, table):
    """Determine how many codons code for the same amino acid / are also stop (degeneration degree of the input codon)

    @param codon the codon
    @param table code table id
    @return the number of codons also coding for the amino acid codon codes for

    """
    return len(altcodons(codon, table))

In [11]:
def define_table():
    """Define the table that maps each amino acid to its synonymus codons (without stop codons).
    @return a dictionary, where the keys are amino acids and the values are the codons.
    
    """
    
    standard_table = CodonTable.unambiguous_dna_by_name["Standard"]
    reverse_genetic_code = standard_table.back_table.copy()

    for aa in reverse_genetic_code.keys():
        curr_amino_acid = aa
        primary_codon = reverse_genetic_code[aa]
        syno_codons = altcodons(primary_codon, 1)
        reverse_genetic_code[curr_amino_acid] = syno_codons

    reverse_genetic_code.pop(None, None)
    return reverse_genetic_code

In [12]:
def count_codons(gene_seq):
    error_dict = CodonsDict.copy() #empty dictionary
    error_dict = {x:np.NaN for x in error_dict}
    counts = CodonsDict.copy()
    b = len(gene_seq)%3
    for i in range(0,len(gene_seq)-b,3):
        codon = gene_seq[i:i+3]
        counts[codon] += 1
    return counts

In [13]:
def codon_relative_freq(gene_seq):
    """For each codon, the function calculates its relative frequency,
    i.e the number of apperances of the codon,
    divided by the number of appereances of the amino-acid that the codon encodes.
    if the codon is not in the sequence, its relative frequency is set to NaN.
    
    @param gene_seq the sequence of the gene (not string type!)
    @return a dictionary where the keys are codons and the values are the relative frquencies.

    """
    
    counts = count_codons(gene_seq)
    gene_seq
    genetic_code = unambiguous_dna_by_id[1].forward_table #keys = codons; values = Amino acids
    x = {'TGA':'*','TAA':'*','TAG':'*'}
    genetic_code.update(x)
    reverse_genetic_code = define_table() #keys = Amino acids, values = codons
    protein_seq = Seq(gene_seq).translate() 
    return {codon: float(counts[codon])/protein_seq.count(genetic_code[codon]) if protein_seq.count(genetic_code[codon])!=0 else 0 
            for codon in counts.keys()}

In [14]:
# given the output of the function above (def codon_relative_freq(gene_seq)), this function returns a 
def crf_vec(crf):
    d = []
    for key,value in crf.items():
        d.append(value)
    return d

In [15]:
# the function calculates the frequency of each amino acid given a sequence
def AA_freq(gene_seq):
    AA = ['F','L','S','Y','C','W','P','M','Q','H','I','R','K','N','T','V','A','E','G','D','*']
    aa_freq = []
    for aa in AA:
        aa_freq.append((gene_seq.count(aa))/len(gene_seq))
    return aa_freq

In [16]:
# this function return the distances from the target gene to essential gene for the profiles of codon relative frequency and amino acid relative frequency
# in addition, the distances from the target gene to essential gene closet genes for the profiles of codon relative frequency and amino acid relative frequency
# paramters required: essential gene sequence[i], target gene sequence, df2.iloc[i] (the coresponding row in df2 of the chosen essential gene),all yeast genes sequence
# _essen means a distance from eseential gene
# _close means a distance from eseential gene closest genes
# _aa means distance for amino acis profile
# _c means distance for codons profile

def calc_dis_near_genes(gene_sequence,target_gene,df2,genes_sequence):
    
    aaf = np.asarray(AA_freq(gene_seq_2_aa(target_gene)))
    crf = np.asarray(crf_vec(codon_relative_freq(target_gene.upper())))
    gene_seq_aaf = np.asarray(AA_freq(gene_seq_2_aa(gene_sequence)))
    gene_seq_crf = np.asarray(crf_vec(codon_relative_freq(gene_sequence)))
    
    euc_dist_c_essen = distance.euclidean(crf,gene_seq_crf)
    cart_d_c_essen = sum(crf*gene_seq_crf)
    corr_p_c_essen = pearsonr(crf,gene_seq_crf)[0]
    corr_s_c_essen = stats.spearmanr(crf,gene_seq_crf)[0]
    L1_dist_c_essen = sum(np.abs(crf-gene_seq_crf))
    ks_dist_c_essen = ks_2samp(crf,gene_seq_crf)[0]
    
    euc_dist_aa_essen = distance.euclidean(aaf,gene_seq_aaf)
    cart_d_aa_essen = sum(aaf*gene_seq_aaf)
    corr_p_aa_essen = pearsonr(aaf,gene_seq_aaf)[0]
    corr_s_aa_essen = stats.spearmanr(aaf,gene_seq_aaf)[0]
    L1_dist_aa_essen = sum(np.abs(aaf-gene_seq_aaf))
    ks_dist_aa_essen = ks_2samp(aaf,gene_seq_aaf)[0]
    
    max_5_ind_1 = df2['max_5_ind_1']
    max_5_ind_2 = df2['max_5_ind_2']
    max_5_ind_3 = df2['max_5_ind_3']
    max_5_ind_4 = df2['max_5_ind_4']
    max_5_ind_5 = df2['max_5_ind_5']
    
    if max_5_ind_1 == -2:
        
        euc_dist_c_close = []
        cart_d_c_close = []
        corr_p_c_close = []
        corr_s_c_close = []
        L1_dist_c_close = []
        ks_dist_c_close = []
    
        euc_dist_aa_close = []
        cart_d_aa_close = []
        corr_p_aa_close = []
        corr_s_aa_close = []
        L1_dist_aa_close = []
        ks_dist_aa_close = []
    else:
        ind = [max_5_ind_1,max_5_ind_2,max_5_ind_3,max_5_ind_4,max_5_ind_5]
        for index in ind:
            if index==-2:
                ind.remove(index)
        close_genes_c = []
        close_genes_aa = []
        for j in range(0,len(ind)):
            index = ind[j]
            close_genes_c.append(crf_vec(codon_relative_freq(genes_sequence[index])))
            close_genes_aa.append(AA_freq(gene_seq_2_aa(genes_sequence[index])))
        m_close_genes_c = np.mean(close_genes_c,axis=0)
        m_close_genes_aa = np.mean(close_genes_aa,axis=0)
    
        euc_dist_c_close = distance.euclidean(crf,m_close_genes_c)
        cart_d_c_close = sum(crf*m_close_genes_c)
        corr_p_c_close = pearsonr(crf,m_close_genes_c)[0]
        corr_s_c_close = stats.spearmanr(crf,m_close_genes_c)[0]
        L1_dist_c_close = sum(np.abs(crf-m_close_genes_c))
        ks_dist_c_close = ks_2samp(crf,m_close_genes_c)[0]
        
        euc_dist_aa_close = distance.euclidean(aaf,m_close_genes_aa)
        cart_d_aa_close = sum(aaf*m_close_genes_aa)
        corr_p_aa_close = pearsonr(aaf,m_close_genes_aa)[0]
        corr_s_aa_close = stats.spearmanr(aaf,m_close_genes_aa)[0]
        L1_dist_aa_close = sum(np.abs(aaf-m_close_genes_aa))
        ks_dist_aa_close = ks_2samp(aaf,m_close_genes_aa)[0]
          
    return(euc_dist_c_essen,corr_p_c_essen,corr_s_c_essen,ks_dist_c_essen,cart_d_c_essen,L1_dist_c_essen,euc_dist_aa_essen,corr_p_aa_essen,corr_s_aa_essen,ks_dist_aa_essen,cart_d_aa_essen,L1_dist_aa_essen,euc_dist_c_close,corr_p_c_close,corr_s_c_close,ks_dist_c_close,cart_d_c_close,L1_dist_c_close,euc_dist_aa_close,corr_p_aa_close,corr_s_aa_close,ks_dist_aa_close,cart_d_aa_close,L1_dist_aa_close)


In [17]:
# the function calculates the folding energy profile in a window of 40 nucletides each time and a shift of 10 nt (150 nt window in tot).
# if the sequence length is less than 300 the rest of the profile will be filll with the average of the gc content of existing windows 
# ind -1 for 300 nt from the beginning of the sequenece (for essential genes) or 1 for 300 until the end of the sequenece (for target gene)
def fe_profile(gene_seq,ind):
    fe_profile = []
    i = 0
    j = 0
    if len(gene_seq)>150:
        if ind == -1:
            wind = gene_seq[0:150]
            while j < 12:
                (ss, mfe) = RNA.fold(wind[i:i+40])
                fe_content_profile.append(mfe)
                i = i + 10
                j = j + 1
        if ind == 1:
            wind = gene_seq[len(gene_seq)-150:]
            while j < 12:
                (ss, mfe) = RNA.fold(wind[i:i+40])
                fe_content_profile.append(mfe)
                i = i + 10
                j = j + 1
    else:
        if ind == -1:
            x = (math.floor(len(gene_seq)/10))-3
            j = 0
            for i in range(0,x):
                (ss, mfe) = RNA.fold(wind[j:j+40])
                fe_content_profile.append(mfe)
                j = j + 10
            m = mean(fe_content_profile)
            for i in range(x,12):
                fe_content_profile.append(m)
        if ind == 1:
            x = (math.floor(len(gene_seq)/10))-3
            j = 0
            d = len(gene_seq)%10
            wind = gene_seq[d:]
            for i in range(0,x):
                (ss, mfe) = RNA.fold(wind[j:j+40])
                fe_content_profile.append(mfe)
                j = j + 10
            m = mean(fe_content_profile)
            for i in range(x,12):
                fe_content_profile.append(m)        
    return fe_content_profile

In [18]:
# calculates ditances of folding energy profiles from target gene to essential gene, and to its closest gene
# returns 6 distances from essential gene and 6 from its closet genes (from the avergae on their profiles)
# paramters required: essential gene sequence[i], target gene sequence, df2.iloc[i] (the coresponding row in df2 of the chosen essential gene),all yeast genes sequence
# _essen mean a distance from eseential gene
# _close mean a distance from eseential gene closest genes

def calc_dist_fe(gene_sequence,target_gene,df2,genes_sequence):
    
    fe_target_gene = np.asarray(fe_profile(target_gene,1))
    fe_gene = np.asarray(fe_profile(gene_sequence,-1))
    
    euc_dist_fe_essen = distance.euclidean(fe_target_gene,fe_gene)
    cart_d_fe_essen = sum(fe_target_gene*fe_gene)
    corr_p_fe_essen = pearsonr(fe_target_gene,fe_gene)[0]
    corr_s_fe_essen = stats.spearmanr(fe_target_gene,fe_gene)[0]
    L1_dist_fe_essen = sum(np.abs(fe_target_gene-fe_gene))
    ks_dist_fe_essen = ks_2samp(fe_target_gene,fe_gene)[0]
    
    max_5_ind_1 = df2['max_5_ind_1']
    max_5_ind_2 = df2['max_5_ind_2']
    max_5_ind_3 = df2['max_5_ind_3']
    max_5_ind_4 = df2['max_5_ind_4']
    max_5_ind_5 = df2['max_5_ind_5']
    
    if max_5_ind_1 == -2:
        euc_dist_fe_close = []
        corr_p_fe_close = []
        corr_s_fe_close = []
        ks_dist_fe_close = []
        cart_d_fe_close = []
        L1_dist_fe_close = []
    else:
        ind = [max_5_ind_1,max_5_ind_2,max_5_ind_3,max_5_ind_4,max_5_ind_5]
        for index in ind:
            if index==-2:
                ind.remove(index)
        close_genes_fe = []
        for j in range(0,len(ind)):
            index = ind[j]
            close_genes_fe.append(fe_profile(genes_sequence[index],-1))
        m_close_genes_fe = np.mean(close_genes_fe,axis=0)
    
        euc_dist_fe_close = distance.euclidean(fe_target_gene,m_close_genes_fe)
        cart_d_fe_close = sum(fe_target_gene*m_close_genes_fe)
        corr_p_fe_close = pearsonr(fe_target_gene,m_close_genes_fe)[0]
        corr_s_fe_close = stats.spearmanr(fe_target_gene,m_close_genes_fe)[0]
        L1_dist_fe_close = sum(np.abs(fe_target_gene-m_close_genes_fe))
        ks_dist_fe_close = ks_2samp(fe_target_gene,m_close_genes_fe)[0]
         
    return(euc_dist_fe_essen,corr_p_fe_essen,corr_s_fe_essen,ks_dist_fe_essen,cart_d_fe_essen,L1_dist_fe_essen,euc_dist_fe_close,corr_p_fe_close,corr_s_fe_close,ks_dist_fe_close,cart_d_fe_close,L1_dist_fe_close)  